In [1]:
import pandas as pd
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from google.cloud import translate_v2 as translate
from tqdm import tqdm
import string
import preprocessor as p
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/82106/Desktop/key/innate-vigil-349313-c1b81308a813.json'

In [4]:
file_name = input('file name : ')

file name : 2022-01-01_2022-05-09_league of legend_tweet_2000


In [16]:
# 원본 파일 읽어옴
df = pd.read_csv('./corpus/tweet/' + file_name + '.csv', index_col=0, encoding="UTF-8")
df.drop_duplicates(subset=['Text'], inplace=True)

In [17]:
pd.set_option('display.max_colwidth', None)
pd.DataFrame(df['Text'])

,Text
0,I was the league of legends guy https://t.co/JuStIXRvgn
1,Hoje uma turma ai voltou pro league of legends e nunca mais vai falar de futebol na vida né
2,"The most entertaining league of legends content is when people get super tilted and angry. IE: Tyler1 explaining how bad his teamates are is funny asf, SRO has the funniest rage montage, valkin flaming humzh and i think that says a lot about league"
3,"@Shadowjay1431 @RaikonedPT @LeagueOfLegends Yes, the tower is the safest place in lane. But it is not completely infallible and I don't think it should be. I don't think saying that is asinine.\n\nYes, you can back off a turret. Sacrificing waves to deny kill/assist gold is not the end of the world."
4,We need to collectively obliterate league of legends from our society
...,...
1996,"04:03: ""Enemy"" von Imagine Dragons feat. JID &amp; League Of Legends"
1997,@Samandriade @LeagueOfLegends @FruitfulMage @StoneTaliyah @PepprNSalt
1998,"Goin to be playing ARAMs in League of Legends tonight for \nMEME\nSTREAM\nREDEEM\n\nSo please offer me some emotional support, I am about to have my booty slamdanced. ;w; \n\n⭐️Where: Fawnlace_ on TTV\n⭐️When: Right meow\n ⭐️Who: Me, you, and the Stardust crew!\n\n✨Starting in 5 minutes!✨"
1999,🚨🚨🚨NOTÍCIA DE ÚLTIMA HORA! Hank teve sua conta de League of Legends hackeada! https://t.co/esvGitXSUj


In [18]:
# 전처리 함수 내 사용
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "&gt;" : "is greater than", "&lt;" : "is less than", "&nbsp;": " ", "&amp;" : "and", "&quot;" : '"'}

In [19]:
def preprocessing(data) :
    data = re.sub('’|‘|´|`', '\'', data)
    # normalize abbreviation
    data = ' '.join([contractions[t] if t in contractions else t for t in data.split(" ")])
    # clean corpus to remove emoji and URL
    data = p.clean(data)
    # remove numbers and punctuations
    data = re.sub(r'[^A-Za-z ]', ' ', data)
    # lowering
    data = data.lower()
    # remove duplicated alphabets
    data = re.sub(r'([A-Za-z])\1{2,}', r'\1', data)
    
    data = re.sub('league of legends', 'leagueoflegends', data)
    
    return data

In [20]:
df['Cleaned'] = df['Text'].apply(lambda x : preprocessing(x))
df[['Text', 'Cleaned']]

,Text,Cleaned
0,I was the league of legends guy https://t.co/JuStIXRvgn,i was the leagueoflegends guy
1,Hoje uma turma ai voltou pro league of legends e nunca mais vai falar de futebol na vida né,hoje uma turma ai voltou pro leagueoflegends e nunca mais vai falar de futebol na vida n
2,"The most entertaining league of legends content is when people get super tilted and angry. IE: Tyler1 explaining how bad his teamates are is funny asf, SRO has the funniest rage montage, valkin flaming humzh and i think that says a lot about league",the most entertaining leagueoflegends content is when people get super tilted and angry ie tyler explaining how bad his teamates are is funny asf sro has the funniest rage montage valkin flaming humzh and i think that says a lot about league
3,"@Shadowjay1431 @RaikonedPT @LeagueOfLegends Yes, the tower is the safest place in lane. But it is not completely infallible and I don't think it should be. I don't think saying that is asinine.\n\nYes, you can back off a turret. Sacrificing waves to deny kill/assist gold is not the end of the world.",yes the tower is the safest place in lane but it is not completely infallible and i do not think it should be i do not think saying that is asinine yes you can back off a turret sacrificing waves to deny kill assist gold is not the end of the world
4,We need to collectively obliterate league of legends from our society,we need to collectively obliterate leagueoflegends from our society
...,...,...
1996,"04:03: ""Enemy"" von Imagine Dragons feat. JID &amp; League Of Legends",enemy von imagine dragons feat jid and leagueoflegends
1997,@Samandriade @LeagueOfLegends @FruitfulMage @StoneTaliyah @PepprNSalt,
1998,"Goin to be playing ARAMs in League of Legends tonight for \nMEME\nSTREAM\nREDEEM\n\nSo please offer me some emotional support, I am about to have my booty slamdanced. ;w; \n\n⭐️Where: Fawnlace_ on TTV\n⭐️When: Right meow\n ⭐️Who: Me, you, and the Stardust crew!\n\n✨Starting in 5 minutes!✨",goin to be playing arams in leagueoflegends tonight for memestreamredeemso please offer me some emotional support i am about to have my booty slamdanced w where fawnlace on ttvwhen right meow who me you and the stardust crew starting in minutes
1999,🚨🚨🚨NOTÍCIA DE ÚLTIMA HORA! Hank teve sua conta de League of Legends hackeada! https://t.co/esvGitXSUj,notcia de ltima hora hank teve sua conta de leagueoflegends hackeada


In [21]:
client = translate.Client()

def is_english(data) :
    if len(data) < 2: return ''
    if client.detect_language(data)['language'] == 'en': return data
    else : return ''

df['Cleaned'] = df['Cleaned'].apply(lambda x : is_english(x))

In [22]:
df[['Text', 'Cleaned']]

,Text,Cleaned
0,I was the league of legends guy https://t.co/JuStIXRvgn,i was the leagueoflegends guy
1,Hoje uma turma ai voltou pro league of legends e nunca mais vai falar de futebol na vida né,
2,"The most entertaining league of legends content is when people get super tilted and angry. IE: Tyler1 explaining how bad his teamates are is funny asf, SRO has the funniest rage montage, valkin flaming humzh and i think that says a lot about league",the most entertaining leagueoflegends content is when people get super tilted and angry ie tyler explaining how bad his teamates are is funny asf sro has the funniest rage montage valkin flaming humzh and i think that says a lot about league
3,"@Shadowjay1431 @RaikonedPT @LeagueOfLegends Yes, the tower is the safest place in lane. But it is not completely infallible and I don't think it should be. I don't think saying that is asinine.\n\nYes, you can back off a turret. Sacrificing waves to deny kill/assist gold is not the end of the world.",yes the tower is the safest place in lane but it is not completely infallible and i do not think it should be i do not think saying that is asinine yes you can back off a turret sacrificing waves to deny kill assist gold is not the end of the world
4,We need to collectively obliterate league of legends from our society,we need to collectively obliterate leagueoflegends from our society
...,...,...
1996,"04:03: ""Enemy"" von Imagine Dragons feat. JID &amp; League Of Legends",enemy von imagine dragons feat jid and leagueoflegends
1997,@Samandriade @LeagueOfLegends @FruitfulMage @StoneTaliyah @PepprNSalt,
1998,"Goin to be playing ARAMs in League of Legends tonight for \nMEME\nSTREAM\nREDEEM\n\nSo please offer me some emotional support, I am about to have my booty slamdanced. ;w; \n\n⭐️Where: Fawnlace_ on TTV\n⭐️When: Right meow\n ⭐️Who: Me, you, and the Stardust crew!\n\n✨Starting in 5 minutes!✨",goin to be playing arams in leagueoflegends tonight for memestreamredeemso please offer me some emotional support i am about to have my booty slamdanced w where fawnlace on ttvwhen right meow who me you and the stardust crew starting in minutes
1999,🚨🚨🚨NOTÍCIA DE ÚLTIMA HORA! Hank teve sua conta de League of Legends hackeada! https://t.co/esvGitXSUj,


In [44]:
df_temp = df[df['Cleaned'] == ''].index
df = df.drop(df_temp)
result = df[['Datetime', 'Cleaned']].rename(columns={'Cleaned':'Text'})
result['Text'] = result['Text'].apply(lambda x : ' '.join(x.split()))
result

,Datetime,Text
0,2022-05-08,i was the leagueoflegends guy
2,2022-05-08,the most entertaining leagueoflegends content is when people get super tilted and angry ie tyler explaining how bad his teamates are is funny asf sro has the funniest rage montage valkin flaming humzh and i think that says a lot about league
3,2022-05-08,yes the tower is the safest place in lane but it is not completely infallible and i do not think it should be i do not think saying that is asinine yes you can back off a turret sacrificing waves to deny kill assist gold is not the end of the world
4,2022-05-08,we need to collectively obliterate leagueoflegends from our society
6,2022-05-08,potatoise arcane leagueoflegends
...,...,...
1994,2022-05-08,this has spiraled hard into irrelevant topics or everyone talking in circles so i leave this vi masks flirting with insults as proven with caitlyn if we take jayce being called pretty boy at face value she is bi if we take it in a subjective text then she is bi
1995,2022-05-08,the idea leagueoflegends has confirmed lore arcane is an au in the game it is a skin line there are lots of different skin lines attached to various aus the i do not know hub universe lore default skin lore is not remotely fixed minute to minute
1996,2022-05-08,enemy von imagine dragons feat jid and leagueoflegends
1998,2022-05-08,goin to be playing arams in leagueoflegends tonight for memestreamredeemso please offer me some emotional support i am about to have my booty slamdanced w where fawnlace on ttvwhen right meow who me you and the stardust crew starting in minutes


In [45]:
result.reset_index(drop=True).set_index('Datetime').to_csv('sample_preprocessed.csv')

In [46]:
stop_words = set(stopwords.words('english')) 
result['Text'] = result['Text'].apply(lambda x : ' '.join(word for word in x.split() if not word in stop_words if len(word) > 1))
result

,Datetime,Text
0,2022-05-08,leagueoflegends guy
2,2022-05-08,entertaining leagueoflegends content people get super tilted angry ie tyler explaining bad teamates funny asf sro funniest rage montage valkin flaming humzh think says lot league
3,2022-05-08,yes tower safest place lane completely infallible think think saying asinine yes back turret sacrificing waves deny kill assist gold end world
4,2022-05-08,need collectively obliterate leagueoflegends society
6,2022-05-08,potatoise arcane leagueoflegends
...,...,...
1994,2022-05-08,spiraled hard irrelevant topics everyone talking circles leave vi masks flirting insults proven caitlyn take jayce called pretty boy face value bi take subjective text bi
1995,2022-05-08,idea leagueoflegends confirmed lore arcane au game skin line lots different skin lines attached various aus know hub universe lore default skin lore remotely fixed minute minute
1996,2022-05-08,enemy von imagine dragons feat jid leagueoflegends
1998,2022-05-08,goin playing arams leagueoflegends tonight memestreamredeemso please offer emotional support booty slamdanced fawnlace ttvwhen right meow stardust crew starting minutes


In [48]:
result_temp = result[result['Text'] == ''].index
print(result_temp)
result = result.drop(result_temp)
result

Int64Index([17, 150, 234, 441, 1213, 1568], dtype='int64')


,Datetime,Text
0,2022-05-08,leagueoflegends guy
2,2022-05-08,entertaining leagueoflegends content people get super tilted angry ie tyler explaining bad teamates funny asf sro funniest rage montage valkin flaming humzh think says lot league
3,2022-05-08,yes tower safest place lane completely infallible think think saying asinine yes back turret sacrificing waves deny kill assist gold end world
4,2022-05-08,need collectively obliterate leagueoflegends society
6,2022-05-08,potatoise arcane leagueoflegends
...,...,...
1994,2022-05-08,spiraled hard irrelevant topics everyone talking circles leave vi masks flirting insults proven caitlyn take jayce called pretty boy face value bi take subjective text bi
1995,2022-05-08,idea leagueoflegends confirmed lore arcane au game skin line lots different skin lines attached various aus know hub universe lore default skin lore remotely fixed minute minute
1996,2022-05-08,enemy von imagine dragons feat jid leagueoflegends
1998,2022-05-08,goin playing arams leagueoflegends tonight memestreamredeemso please offer emotional support booty slamdanced fawnlace ttvwhen right meow stardust crew starting minutes


In [50]:
result.reset_index(drop=True).set_index('Datetime').to_csv('sample_preprocessed_stopwords.csv')